# Classes and OOP

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
os.getcwd()
os.chdir('/Users/fizz/Document/Notes/Python/codes')

Find the first occurrence of *attribute* by looking in *object*, then in all classes above it, from bottom to top and left to right.

Attribute *reference* kick off inheritance searches, but attribute *assignments* affect only the objects in which the assignments are made.

In [5]:
class rec: pass
rec.name = 'Bob'
rec.age = 40
x = rec()
y = rec()
x.name = 'Sue'
rec.name, x.name, y.name

('Bob', 'Sue', 'Bob')

In [10]:
list(rec.__dict__.keys())
list(name for name in rec.__dict__ if not name.startswith('__'))
list(x.__dict__)
list(y.__dict__.keys())

['__module__', '__dict__', '__weakref__', '__doc__', 'name', 'age']

['name', 'age']

['name']

[]

An attribute can often be fetched by *either* dictionary indexing or attribute notation, but only if it's present on the object in question - attribute notation kicks off inheritance search, but indexing looks in the single object only.

In [13]:
# Each instance has a link to its class
x.__class__
# Classes have a __bases__ attribute, which is a tuple of references to their
# superclass objects
rec.__bases__

def upper(obj):
    return obj.name.upper()
rec.method = upper
x.method()
rec.method(x)

__main__.rec

(object,)

'SUE'

'SUE'

In [ ]:
# person.py
class Person:
    def __init__(self, name, job=None, pay=0):
        self.name = name
        self.job = job
        self.pay = pay
    def lastName(self):
        return self.name.split()[-1]
    def giveRaise(self, percent):
        self.pay = int(self.pay * (1 + percent))
    def __repr__(self):
        return '[Person: %s, %s]' % (self.name, self.pay)

class Manager(Person):
    def __init__(self, name, pay):
        Person.__init__(self, name, 'mgr', pay)
    def giveRaise(self, percent, bonus=0.10):
        person.giveRaise(self, percent + bonus)

if __name__ == '__main__':
    bob = Person('Bob Smith')
    sue = Person('Sue Jones', job='dev', pay=100000)
    print(bob)
    print(sue)
    print(bob.lastName(), sue.lastName())
    sue.giveRaise(.10)
    print(sue)
    tom = Manager('Tom Jones', 50000)
    tom.giveRaise(.10)
    print(tom.lastName())
    print(tom)

Technically, **\_\_str\_\_** is preferred by **print** and **str**, and **\_\_repr\_\_** is used as a fallback for these roles and in all other contexts.

Calling through the class directly effectively subverts inheritance and kicks the call higher up the class tree to run a specific version. In our case, we can use this technique to invoke the default **giveRaise** in **Person**, even though it's been redefined at the **Manager**'s **giveRaise** code would loop - since **self** already is a **Manager**, **self.giveRaise()** would resolve again to **Manager.giveRaise**, and so on and so forth *recursively* until available memory is exhausted.

In [ ]:
# Embedding-based Manager alternative
class Person:
    ...same...
class Manager:
    def __init__(self, name, pay):
        self.person = Person(name, 'mgr', pay)
    def giveRaise(self, percent, bouns=.10):
        self.person.giveRaise(percent + bouns)
    def __getattr__(self, attr):
        return getattr(self.person, attr)
    def __repr__(self):
        return str(self.person)
class Department:
    def __init__(self, *args):
        self.members = list(args)
    def addMember(self, person):
        self.members.append(person)
    def giveRaise(self, percent):
        for person in self.members:
            person.giveRaise(percent)
    def showAll(self):
        for person in self.members:
            print(person)
if __name__ == '__main__':
    ...same...
# This alternative is representative of a general coding pattern usually known
# as DELEGATION - a composite-based structure that manages a wrapped
# object and propagates method calls to it.

Recall that built-in operations like printing and addition implicitly invoke operator overloading methods such as **\_\_repr\_\_** and **\_\_add\_\_**. In 3.X's new-style classes, built-in operations like these do not route their implicit attribute fetches through generic attribute managers: neither **\_\_getattr\_\_** (run for undefined attributes) nor its cousin **\_\_getattribute\_\_** (run for all attributes) is invoked. This is why we have to redefine **\_\_repr\_\_** redundantly in the alternative **Manager**, in order to ensure that printing is routed to the embedded **Person** object in 3.X.

In [9]:
from person import Person
bob = Person('Bob Smith')
bob
bob.__class__.__name__
list(bob.__dict__.keys())
for key in bob.__dict__:
    print(key, '=>', getattr(bob, key))

[Person: Bob Smith, 0]

'Person'

['name', 'job', 'pay']

name => Bob Smith
job => None
pay => 0


In [14]:
# File classtools.py (new)
"Assorted class utilities and tools"

class AttrDisplay:
    """
    Provides an inheritable display overload method that shows instances with their class names
    and a name=value pair for each attribute stored on the instance itself (but not attrs inherited
    from its classes). Can be mixed into any class, and will work on any instance.
    """
    def gatherAttrs(self):
        attrs =[]
        for key in sorted(self.__dict__):
            attrs.append('%s=%s' % (key, getattr(self, key)))
        return ', '.join(attrs)
    
    def __repr__(self):
        return '[%s: %s]' % (self.__class__.__name__, self.gatherAttrs())
    
if __name__ == '__main__':
    class TopTest(AttrDisplay):
        count = 0
        def __init__(self):
            self.attr1 = TopTest.count
            self.attr2 = TopTest.count + 1
            TopTest.count += 2
    class SubTest(TopTest):
        pass
    
    X, Y = TopTest(), SubTest()
    print(X)
    print(Y)

'Assorted class utilities and tools'

[TopTest: attr1=0, attr2=1]
[SubTest: attr1=2, attr2=3]


In [17]:
list(bob.__dict__.keys())
dir(bob)
list(name for name in dir(bob) if not name.startswith('__'))

['name', 'job', 'pay']

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'giveRaise',
 'job',
 'lastName',
 'name',
 'pay']

['giveRaise', 'job', 'lastName', 'name', 'pay']

Storing Objects in a Database

1. pickle: Serializes arbitrary Python objects to and from a string of bytes

2. dbm: Implements an access-by-key filesystem for storing strings

3. shelve: Uses the other two modules to store Python objects on a file by key

In [19]:
from person import Person, Manager
bob = Person('Bob Smith')
sue = Person('Sue Jones', job='dev', pay=100000)
tom = Manager('Tom Jones', 50000)

import shelve
db = shelve.open('persondb')
for obj in (bob, sue, tom):
    db[obj.name] = obj
db.close()

In [23]:
# glob module allows us to get directory listings in Python code to verify the files here
import glob
glob.glob('person*')

# the interactive prompt effectively becomes a database client
import shelve
db = shelve.open('persondb')
len(db)
list(db.keys())
for key in sorted(db):
    print(key, '=>', db[key])

['person.py', 'persondb.db']

3

['Bob Smith', 'Sue Jones', 'Tom Jones']

Bob Smith => [Person: Bob Smith, 0]
Sue Jones => [Person: Sue Jones, 100000]
Tom Jones => [Person: Tom Jones, 50000]


Notice that we don't have to import our **Person** or **Manager** classes here in order to load or use our stored objects. This works because when Python pickles a class instance, it records its **self** instance attributes, along with the name of the class it was created from and the module where the class lives. When **bob** is later fetched from the shelve and unpickled, Python will automatically reimport the class and link **bob** to it.